# Dev Data Slope Fitting

Note: this notebook uses py3.5 (for MNE)

In [2]:
%matplotlib qt
%matplotlib inline

In [3]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt

import mne

In [19]:
from slf.core.db import SLFDB

In [22]:
# Get project database objects, and list of available subjects
db = SLFDB()
subjs = db.check_subjs()

In [ ]:
#''

In [ ]:
db.

In [ ]:
file_name = '/Users/thomasdonoghue/Desktop/EEG_Dat/Subjs/A00051826/EEG/preprocessed/csv_format/gip_A00051826005.csv'
dat = np.loadtxt(file_name, delimiter=',')

In [ ]:
f2 = '/Users/thomasdonoghue/Desktop/EEG_Dat/Subjs/A00054647007/EEG/preprocessed/csv_format/bp_A00054647007.csv'
dat = np.loadtxt(file_name, delimiter=',')

In [ ]:
dat.shape

In [ ]:
# Read default montage
montage = mne.channels.read_montage('GSN-HydroCel-129')
keep_chans = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 
              21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
              38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 50, 51, 52, 53, 54, 55, 57,
              58, 59, 60, 61, 62, 64, 65, 66, 67, 69, 70, 71, 72, 74, 75, 76, 77,
              78, 79, 80, 82, 83, 84, 85, 86, 87, 89, 90, 91, 92, 93, 95, 96, 97,
              98, 100, 101, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 114,
              115, 116, 117, 118, 120, 121, 122, 123, 124, 129])

# Fix zero based indexing (screw off matlab)
keep_chans = keep_chans - 1

# Get channel names to keep
kept_chans = list(np.array(montage.ch_names)[keep_chans])

In [ ]:
# Settings for loading data
s_freq = 500

In [ ]:
# Create the info structure needed by MNE
info = mne.create_info(kept_chans, s_freq, 'eeg')

# Create the MNE Raw data object
raw = mne.io.RawArray(dat, info)

In [ ]:
# Create a stim channel
stim_info = mne.create_info(['stim'], s_freq, 'stim')
stim_raw = mne.io.RawArray(np.zeros(shape=[1, 103526]), stim_info)

In [ ]:
# Add stim channel to data object
raw.add_channels([stim_raw], force_update_info=True)

In [ ]:
# Check out some data
raw.plot(start=100, duration=2, scalings=dict(eeg=10))

In [ ]:
# Set bad channels
raw.info['bads'] = ['FidT9', 'E6', 'E12', 'E19', 'E23', 'E30']

In [ ]:
# Check PSD
raw.plot_psd()

In [ ]:
# Calculate PSDs
psds, freqs = mne.time_frequency.psd_welch(raw, fmin=1., fmax=40, n_fft=1000)

In [ ]:
# Get freq res of PSD
f_res = np.mean(np.diff(freqs))

In [ ]:
plt.plot(np.log10(freqs), np.log10(psds[30, :]))

In [ ]:
# Set the event file name
ev_f_name = ("/Users/thomasdonoghue/Desktop/EEG_Dat/Subjs/A00051826/EEG/"
            "preprocessed/csv_format/gip_A00051826005_events.csv")

In [ ]:
# Load events

# 
headers = ['type', 'value', 'latency', 'duration', 'urevent']
evs = np.empty(shape=[0, 3])

# Load events from csv file
with open(ev_f_name, 'r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:

        # Skip the empty rows
        if row == []: continue
        
        # Skip the header row FOR EVERY DAMN EVENT. SERIOUSLY?
        if row[0] == 'type': continue
        
        evs = np.vstack((evs, np.array([int(row[2]), 0, int(row[0])])))

In [ ]:
# Add events to data object
raw.add_events(evs, stim_channel='stim')